In [66]:
import requests
from selenium import webdriver
from webdriver_manager.utils import chrome_version
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import pandas as pd
import chromedriver_binary
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np 

# 各種設定

In [67]:
op = Options()
# --headlessだけではOSによって動かない、プロキシが弾かれる、
# CUI用の省略されたHTMLが帰ってくるなどの障害が出ます。
# 長いですが、これら6行あって最強かつどんな環境でも動きますので、必ず抜かさないようにしてください。
op.add_argument("no-sandbox")
op.add_argument("--disable-extensions")
# op.add_argument("--headless")
op.add_argument('--disable-gpu')  
op.add_argument('--ignore-certificate-errors')
op.add_argument('--allow-running-insecure-content')
op.add_argument('--disable-web-security')
op.add_argument('--disable-desktop-notifications')
op.add_argument("--disable-extensions")
op.add_argument('--lang=ja')
# op.add_argument('--blink-settings=imagesEnabled=false')
op.add_argument('--disable-dev-shm-usage')
op.add_argument('--proxy-server="direct://"')
op.add_argument('--proxy-bypass-list=*')    
op.add_argument('--start-maximized')

In [68]:
url = "https://jinzai.hellowork.mhlw.go.jp/JinzaiWeb/GICB101010.do?screenId=GICB101010&action=initDisp"

# ログイン操作

In [69]:
driver = webdriver.Chrome(chrome_options=op)
driver.get(url)
time.sleep(5)

#派遣業者
haken = driver.find_element_by_xpath('//*[@id="button_area"]/dt[1]/a')
haken.click()
time.sleep(5)

#各種チェックボックス

roudo = driver.find_element_by_id("ID_cbJigyoshoKbnHan1")
tokutei = driver.find_element_by_id("ID_cbJigyoshoKbnChi1")
zenkoku = driver.find_element_by_id("ID_cbZenkoku1")
check = [roudo,tokutei,zenkoku]

for i in check:
    i.click()
    

# 検索
search = driver.find_element_by_id("id_btnSearch")
search.click()

time.sleep(5)

/Users/ryusei/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:1: DeprecationWarning: use options instead of chrome_options
  """Entry point for launching an IPython kernel.


# クローリングの操作の関数

In [113]:
#カラム
tt = ["許可・受理番号\n／許可年月日・\n届出受理年月日", "事業主名称/\n事業所名称","会社url", "事業所所在地/\n電話番号", "得意とする職種", "労使協定の締結", '各種情報', '備考']

In [74]:
#空のdataframeの作製 はじめにconcatとしておけば後もこの名前に上書き処理で行ける
concat = pd.DataFrame(index=[], columns=tt)

In [75]:
tp= driver.find_elements_by_xpath('//*[@id="search"]/tbody/tr')

In [104]:
tc = tp[1].find_elements_by_tag_name("td")

In [107]:
judge_a =  tc[1].find_element_by_tag_name("a")

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"a"}
  (Session info: chrome=87.0.4280.88)


In [106]:
judge_a.get_attribute("href")

"javascript:doPostAction('detail','%E6%B4%BE01-010028,0     ')"

In [114]:
# 1ページ分のループ
def table_loop(driver,col):
    #テーブルの親要素 21の要素　始めの一つはcolmun
    tp= driver.find_elements_by_xpath('//*[@id="search"]/tbody/tr')
    list_fin=[]
    #j=1のときにaタグがあるか選択する
    for i,j in enumerate(tp):
        tc = j.find_elements_by_tag_name("td")
        non=[]
        for k,n in enumerate(tc):
            non.append(n.text)
            #かいしゃの urlがあれば登録する処理
            if k ==1:
                try:
                    non.append(n.find_element_by_tag_name("a").get_attribute("href"))
                except:
                    non.append("")
        list_fin.append(non)
        #ここで一つリストが作成される
    list_fin = pd.DataFrame(list_fin,columns=col).iloc[1:,:]
    return list_fin

In [115]:
a  = table_loop(driver,tt)

In [116]:
a

,許可・受理番号\n／許可年月日・\n届出受理年月日,事業主名称/\n事業所名称,会社url,事業所所在地/\n電話番号,得意とする職種,労使協定の締結,各種情報,備考
1,派01-010028\n昭和63年09月01日,株式会社ツーリストサービス北海道\n株式会社ツーリストサービス北海道,,北海道札幌市中央区北3条西2-2-1 日通札幌ビル7階\n011-222-6097,,,,
2,派01-010033\n平成01年10月01日,株式会社ケイ・イー・パワースタッフ\n株式会社ケイ・イー・パワースタッフ,,北海道札幌市中央区北一条東一丁目4番地1 サン経成ビル5階\n011-232-4096,,,,
3,派01-010050\n平成03年06月01日,キャリアバンク株式会社\nキャリアバンク株式会社,http://www.career-bank.co.jp/,北海道札幌市中央区北5条西5-7 sapporo55 5F\n011-251-3373,,,,
4,派01-010050\n平成03年06月01日,キャリアバンク株式会社\nキャリアバンク株式会社 函館支店,,北海道函館市五稜郭町1-14 五稜郭114ビル 5階\n0138-55-2002,,,,
5,派01-010050\n平成03年06月01日,キャリアバンク株式会社\nキャリアバンク株式会社 旭川支店,http://www.career-bank.co.jp/,北海道旭川市1条通9-50-3 旭川緑橋第一生命ビル3階\n0166-29-2077,,,,
6,派01-010050\n平成03年06月01日,キャリアバンク株式会社\nキャリアバンク株式会社 帯広支店,,北海道帯広市西2条南11-12-1 天光堂ビル3階\n0155-67-7270,,,,
7,派01-010050\n平成03年06月01日,キャリアバンク株式会社\nキャリアバンク株式会社 盛岡オフィス,http://www.career-bank.co.jp/,岩手県盛岡市大通3丁目2番8号 岩手金属工業会館 3階\n019-601-5528,,,,
8,派01-010050\n平成03年06月01日,キャリアバンク株式会社\nキャリアバンク株式会社 仙台支店,,宮城県仙台市青葉区本町2丁目10番33号 第二日本オフィスビル8階6号室\n022-395-...,,,,
9,派01-010053\n平成03年12月01日,リンケージサービス株式会社\nリンケージサービス株式会社,,北海道札幌市中央区北二条西二丁目15番地 ＳＴＶ北2条ビル2階\n011-272-2234,,,,
10,派01-010053\n平成03年12月01日,リンケージサービス株式会社\nリンケージサービス株式会社 東京支店,,東京都千代田区丸の内三丁目4番1号 新国際ビル9階\n03-6809-1318,,,,


In [65]:
#クリック処理 1loop文
try:
    page = driver.find_elements_by_xpath('//*[@id="search_area"]/table/tbody/tr[5]/td/table[2]/tbody/tr/td/table/tbody/tr/td')
    for i in range(len(page)-2):
        #データの処理
        ll = table_loop(driver,tt)
        concat = pd.concat([concat,ll])
        #データの処理が完了した後クリック動作に移る ここでsleepを入れないとクリックの挙動がおかしくなる
        time.sleep(5)
        page = driver.find_elements_by_xpath('//*[@id="search_area"]/table/tbody/tr[5]/td/table[2]/tbody/tr/td/table/tbody/tr/td')
        #>>ボタンを押すと11のように下一桁が１になるから
        page[i+2].click()
        time.sleep(2)
        print(i+1)
#時間がなくて処理を完了したときに出す。
except KeyboardInterrupt:
    pass
#他のエラーで発動
except:
    pass

1


In [27]:
concat

,許可・受理番号\n／許可年月日・\n届出受理年月日,事業主名称/\n事業所名称,事業所所在地/\n電話番号,得意とする職種,労使協定の締結,各種情報,備考
1,派01-010028\n昭和63年09月01日,株式会社ツーリストサービス北海道\n株式会社ツーリストサービス北海道,北海道札幌市中央区北3条西2-2-1 日通札幌ビル7階\n011-222-6097,,,,
2,派01-010033\n平成01年10月01日,株式会社ケイ・イー・パワースタッフ\n株式会社ケイ・イー・パワースタッフ,北海道札幌市中央区北一条東一丁目4番地1 サン経成ビル5階\n011-232-4096,,,,
3,派01-010050\n平成03年06月01日,キャリアバンク株式会社\nキャリアバンク株式会社,北海道札幌市中央区北5条西5-7 sapporo55 5F\n011-251-3373,,,,
4,派01-010050\n平成03年06月01日,キャリアバンク株式会社\nキャリアバンク株式会社 函館支店,北海道函館市五稜郭町1-14 五稜郭114ビル 5階\n0138-55-2002,,,,
5,派01-010050\n平成03年06月01日,キャリアバンク株式会社\nキャリアバンク株式会社 旭川支店,北海道旭川市1条通9-50-3 旭川緑橋第一生命ビル3階\n0166-29-2077,,,,
...,...,...,...,...,...,...,...
16,派06-300079\n平成24年09月01日,株式会社 ナセル\n株式会社 ナセル,山形県米沢市広幡町成島1159-2\n0238-37-3283,,,,
17,派06-300080\n平成24年09月01日,株式会社 バーンフュージョン\n株式会社 バーンフュージョン,山形県酒田市千石町2-13-10\n0234-21-0113,,,,
18,派06-300080\n平成24年09月01日,株式会社 バーンフュージョン\n株式会社 バーンフュージョン 東北テクニカルセンター,山形県鶴岡市美咲町37-25\n0235-64-1117,,,,
19,派06-300080\n平成24年09月01日,株式会社 バーンフュージョン\n株式会社 バーンフュージョン 米沢営業所,山形県米沢市門東町三丁目3番1号 米沢共立ビル2階\n0238-40-1414,,,,


In [28]:
#Falseが重複なし
print(concat.duplicated(subset='事業主名称/\n事業所名称').value_counts())

False    240
dtype: int64


# あとはほっておくように上で作ったループを処理が終わるまで繰り返す。


 片方100回分以下のクリックを飛ばして1000件目からと1件目からでループさせる

In [52]:
judge=True

In [53]:
while judge==True:
    #これで一回10見聞飛べる
    page = driver.find_elements_by_xpath('//*[@id="search_area"]/table/tbody/tr[5]/td/table[2]/tbody/tr/td/table/tbody/tr/td')
    if int(page[1].text)<=100:
        print("進行度:{}".format(page[1].text))
        page[11].click()
        time.sleep(3)
    elif int(page[1].text)>100:
        print("進行度{},ページネーションを停止します".format(page[1].text))
        judge=False

進行度:41
進行度:51
進行度:61
進行度:71
進行度:81
進行度:91
進行度101,ページネーションを停止します


In [24]:
page = driver.find_elements_by_xpath('//*[@id="search_area"]/table/tbody/tr[5]/td/table[2]/tbody/tr/td/table/tbody/tr/td')

In [25]:
page[1].text

'101'

In [129]:
driver.quit()